In [29]:
import pandas as pd
import json
import re

In [30]:
team_info = pd.read_csv('./data/players_21.csv', sep=',',encoding='utf-8')
team_info = team_info[team_info.long_name.notnull()]
team_info = team_info[team_info.long_name != '']

In [31]:
def remove_special_characters(input_str):
    return re.sub(r'[^a-zA-Z0-9\s]', '', input_str)

# leggi il csv e impora i dati in un dataframe in utf-8
df = pd.read_csv('./data/2021-2022-player-stats.csv', sep=';',encoding='ISO-8859-1')
df["Player"] = df["Player"].str.replace('?', '')
df["Player"] = df["Player"].apply(remove_special_characters)

In [32]:
# example clarifications

text = "Paulo Bruno Exequiel Dybala"
substrings = text.split()

if "Paulo" in substrings and "Dybala" in substrings:
    print(True)
else:
    print(False)


True


In [33]:
team_info['long_name_cleaned'] = team_info['long_name'].apply(remove_special_characters)    
team_info.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,long_name_cleaned
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,Lionel Andrs Messi Cuccittini
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,Cristiano Ronaldo dos Santos Aveiro
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png,Robert Lewandowski
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,Neymar da Silva Santos Jnior
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,Kevin De Bruyne


In [34]:
def find_matching_long_name(player_name):
    substrings = player_name.split()

    def checkEquivalence(row):
        return all(substring in row for substring in substrings)
    
    matching_rows = team_info[team_info['long_name_cleaned'].apply(checkEquivalence)]

    if not matching_rows.empty:
        return matching_rows['short_name'].iloc[0]
    else:
        return player_name
    
df['Player'] = df['Player'].apply(lambda x: find_matching_long_name(x))

In [35]:
unique_club_names1 = team_info["club_name"].unique()

unique_club_names2 = df["Squad"].unique()


print(len(unique_club_names1))
print(len(unique_club_names2))

# other problem: club names are different

df = df.merge(team_info[['short_name', 'club_name']],
              left_on='Player', right_on='short_name', how='left')
df = df.drop('short_name', axis=1)
df = df.drop('Squad', axis=1)
df.rename(columns={'club_name': 'Squad'}, inplace=True)
# problem solved

unique_club_names1 = team_info["club_name"].unique()

unique_club_names2 = df["Squad"].unique()


print(len(unique_club_names1))
print(len(unique_club_names2))


682
98
682
383


In [36]:
df = df.merge(team_info[["short_name", "player_face_url", "club_logo_url","club_name"]],
              left_on=['Player', 'Squad'],
              right_on=['short_name', 'club_name'],
              how='left')

df.drop(columns=['short_name', 'club_name'], inplace=True)
df.rename(columns={"player_face_url": "PlayerFaceUrl", "club_logo_url": "ClubLogoUrl"}, inplace=True)

In [37]:
# count number of player with the same SquadName
# note: the number of player with the same SquadName is > 1 (USEFUL for ffill)
print(df['Squad'].value_counts()[df['Squad'].value_counts() < 2])

Squad
Club Deportivo Jorge Wilstermann    1
Hebei FC                            1
Al Ahli                             1
1. FC Heidenheim 1846               1
SC Heerenveen                       1
                                   ..
Paris FC                            1
Gazişehir Gaziantep F.K.            1
KAS Eupen                           1
Sydney FC                           1
Western Sydney Wanderers            1
Name: count, Length: 151, dtype: int64


In [38]:
df['PlayerFaceUrl'].fillna('https://cdn.sofifa.net/player_0.svg', inplace=True)
# sort by SquadName
df.sort_values(by=['Squad'], inplace=True)
# fill none values on ClubLogoUrl field with the row with the same Squad
df['ClubLogoUrl'].fillna(method='ffill', inplace=True)
# correct: no results == no Squad with only 1 player

C:\Users\franc\AppData\Local\Temp\ipykernel_14508\3857645863.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['ClubLogoUrl'].fillna(method='ffill', inplace=True)


In [39]:
# print a Player that includes González in the Player
print(df[df['Player'].str.contains('González')])

        Rk          Player Nation Pos     Comp   Age  Born  MP  Starts   Min  \
1114  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1119  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1122  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   
1121  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   
1118  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1115  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1111  1033  Diego González    ESP  DF  La Liga  27.0  1995  28      19  1793   
1116  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1117  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1120  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   

      ...  PKwon  PKcon   OG  Recov  AerWon  AerLost  AerWon%  \
1114  ...   0.00   0.00  0.0   8.54    2.44     2.93  

In [40]:
acronyms = {
    "Rk": "Rank of the player",
    "Player": "Player's name",
    "Nation": "Player's nation",
    "Pos": "Player's position",
    "Squad": "Name of the squad the player belongs to",
    "Comp": "League that the squad occupies",
    "Age": "Player's age",
    "Born": "Year of birth of the player",
    "MP": "Matches played by the player",
    "Starts": "Matches started by the player",
    "Min": "Minutes played by the player",
    "90s": "Minutes played divided by 90",
    "Goals": "Goals scored or allowed by the player",
    "Shots": "Total shots taken by the player (excluding penalty kicks)",
    "SoT": "Shots on target by the player (excluding penalty kicks)",
    "SoT%": "Shots on target percentage by the player (excluding penalty kicks)",
    "G/Sh": "Goals per shot by the player",
    "G/SoT": "Goals per shot on target by the player (excluding penalty kicks)",
    "ShoDist": "Average distance from goal of all shots taken by the player (excluding penalty kicks)",
    "ShoFK": "Shots from free kicks by the player",
    "ShoPK": "Penalty kicks made by the player",
    "PasProg": "Completed passes that advance the ball towards the opponent's goal",
    "PasAtt": "Total passes attempted by the player",
    "PasLive": "Live-ball passes attempted by the player",
    "PasDead": "Dead-ball passes attempted by the player",
    "PasFK": "Passes attempted from free kicks by the player",
    "TB": "Completed passes sent between back defenders into open space by the player",
    "Sw": "Passes that travel more than 40 yards of the width of the pitch by the player",
    "PasCrs": "Crosses attempted by the player",
    "TI": "Throw-Ins taken by the player",
    "CK": "Corner kicks taken by the player",
    "CkIn": "Inswinging corner kicks taken by the player",
    "CkOut": "Outswinging corner kicks taken by the player",
    "CkStr": "Straight corner kicks taken by the player",
    "PasCmp": "Passes completed by the player",
    "PasOff": "Offsides by the player",
    "PasBlocks": "Passes blocked by the opponent while the player attempted to pass",
    "SCA": "Shot-creating actions by the player",
    "ScaPassLive": "Live-ball passes leading to a shot attempt by the player",
    "ScaPassDead": "Dead-ball passes leading to a shot attempt by the player",
    "ScaDrib": "Successful dribbles leading to a shot attempt by the player",
    "ScaSh": "Shots leading to another shot attempt by the player",
    "ScaFld": "Fouls drawn leading to a shot attempt by the player",
    "ScaDef": "Defensive actions leading to a shot attempt by the player",
    "GCA": "Goal-creating actions by the player",
    "GcaPassLive": "Live-ball passes leading to a goal by the player",
    "GcaPassDead": "Dead-ball passes leading to a goal by the player",
    "GcaDrib": "Successful dribbles leading to a goal by the player",
    "GcaSh": "Shots leading to another goal-scoring shot by the player",
    "GcaFld": "Fouls drawn leading to a goal by the player",
    "GcaDef": "Defensive actions leading to a goal by the player",
    "Tkl": "Number of players tackled by the player",
    "TklWon": "Tackles in which the tackler's team won possession of the ball by the player",
    "TklDef3rd": "Tackles in the defensive 1/3 by the player",
    "TklMid3rd": "Tackles in the middle 1/3 by the player",
    "TklAtt3rd": "Tackles in the attacking 1/3 by the player",
    "TklDri": "Number of dribblers tackled by the player",
    "TklDriAtt": "Number of times dribbled past plus number of tackles by the player",
    "TklDri%": "Percentage of dribblers tackled by the player",
    "TklDriPast": "Number of times dribbled past by an opposing player against the player",
    "Blocks": "Number of times the player blocked the ball by standing in its path",
    "BlkSh": "Number of times the player blocked a shot by standing in its path",
    "PKatt": "Penalty kicks attempted by the player",
    "PasTotCmp": "Total passes completed by the player",
    "PasTotAtt": "Total passes attempted by the player",
    "PasTotCmp%": "Pass completion percentage by the player",
    "PasTotDist": "Total distance traveled by completed passes in any direction by the player",
    "PasTotPrgDist": "Total distance advanced towards the opponent's goal by completed passes by the player",
    "PasShoCmp": "Passes completed between 5 and 15 yards by the player",
    "PasShoAtt": "Passes attempted between 5 and 15 yards by the player",
    "PasShoCmp%": "Pass completion percentage between 5 and 15 yards by the player",
    "PasMedCmp": "Passes completed between 15 and 30 yards by the player",
    "PasMedAtt": "Passes attempted between 15 and 30 yards by the player",
    "PasMedCmp%": "Pass completion percentage between 15 and 30 yards by the player",
    "PasLonCmp": "Passes completed longer than 30 yards by the player",
    "PasLonAtt": "Passes attempted longer than 30 yards by the player",
    "PasLonCmp%": "Pass completion percentage longer than 30 yards by the player",
    "Assists": "Assists by the player",
    "PasAss": "Passes directly leading to a shot (assisted shots) by the player",
    "Pas3rd": "Completed passes entering the 1/3 of the pitch closest to the goal by the player",
    "PPA": "Completed passes into the 18-yard box by the player",
    "CrsPA": "Completed crosses into the 18-yard box by the player",
    "BlkPass": "Number of times the player blocked a pass by standing in its path",
    "Int": "Interceptions by the player",
    "Tkl+Int": "Sum of players tackled and interceptions by the player",
    "Clr": "Clearances by the player",
    "Err": "Mistakes leading to an opponent's shot by the player",
    "Touches": "Total number of times the player touched the ball",
    "TouDefPen": "Touches in the defensive penalty area by the player",
    "TouDef3rd": "Touches in the defensive 1/3 by the player",
    "TouMid3rd": "Touches in the middle 1/3 by the player",
    "TouAtt3rd": "Touches in the attacking 1/3 by the player",
    "TouAttPen": "Touches in the attacking penalty area by the player",
    "TouLive": "Live-ball touches by the player (excluding corner kicks, free kicks, throw-ins, kick-offs, goal kicks, or penalty kicks)",
    "ToAtt": "Number of attempts to take on defenders while dribbling by the player",
    "ToSuc": "Number of defenders successfully dribbled past by the player",
    "ToSuc%": "Percentage of take-ons successfully completed by the player",
    "ToTkl": "Number of times tackled by a defender during a take-on attempt by the player",
    "ToTkl%": "Percentage of times tackled by a defender during a take-on attempt by the player",
    "Carries": "Number of times the player controlled the ball with their feet",
    "CarTotDist": "Total distance the player moved the ball while controlling it with their feet",
    "CarPrgDist": "Total distance the player moved the ball towards the opponent's goal while controlling it with their feet",
    "CarProg": "Carries that move the ball towards the opponent's goal",
    "Car3rd": "Carries that enter the 1/3 of the pitch closest to the goal",
    "CPA": "Carries into the 18-yard box by the player",
    "CarMis": "Number of times the player failed to gain control of the ball",
    "CarDis": "Number of times the player lost control of the ball after being tackled by an opposing player",
    "Rec": "Number of times the player successfully received a pass",
    "RecProg": "Completed passes that advance the ball towards the opponent's goal",
    "CrdY": "Yellow cards received by the player",
    "CrdR": "Red cards received by the player",
    "2CrdY": "Second yellow card received by the player",
    "Fls": "Fouls committed by the player",
    "Fld": "Fouls drawn by the player",
    "Off": "Offsides by the player",
    "Crs": "Crosses attempted by the player",
    "TklW": "Tackles in which the tackler's team won possession of the ball by the player",
    "PKwon": "Penalty kicks won by the player",
    "PKcon": "Penalty kicks conceded by the player",
    "OG": "Own goals by the player",
    "Recov": "Number of loose balls recovered by the player",
    "AerWon": "Aerials won by the player",
    "AerLost": "Aerials lost by the player",
    "AerWon%": "Percentage of aerials won by the player"
}

# dump this dict into a json
with open('./data/acronyms.json', 'w') as fp:
    json.dump(acronyms, fp)


In [41]:
# stampa i valori distinti di comp
print(df['Pos'].unique())

# stampa la lista delle features
print(list(df.columns))

['GK' 'DF' 'FW' 'MF' 'DFFW' 'MFFW' 'FWMF' 'DFMF' 'MFDF' 'FWDF' 'GKMF']
['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Goals', 'Shots', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'ShoDist', 'ShoFK', 'ShoPK', 'PKatt', 'PasTotCmp', 'PasTotAtt', 'PasTotCmp%', 'PasTotDist', 'PasTotPrgDist', 'PasShoCmp', 'PasShoAtt', 'PasShoCmp%', 'PasMedCmp', 'PasMedAtt', 'PasMedCmp%', 'PasLonCmp', 'PasLonAtt', 'PasLonCmp%', 'Assists', 'PasAss', 'Pas3rd', 'PPA', 'CrsPA', 'PasProg', 'PasAtt', 'PasLive', 'PasDead', 'PasFK', 'TB', 'PasPress', 'Sw', 'PasCrs', 'CK', 'CkIn', 'CkOut', 'CkStr', 'PasGround', 'PasLow', 'PasHigh', 'PaswLeft', 'PaswRight', 'PaswHead', 'TI', 'PaswOther', 'PasCmp', 'PasOff', 'PasOut', 'PasInt', 'PasBlocks', 'SCA', 'ScaPassLive', 'ScaPassDead', 'ScaDrib', 'ScaSh', 'ScaFld', 'ScaDef', 'GCA', 'GcaPassLive', 'GcaPassDead', 'GcaDrib', 'GcaSh', 'GcaFld', 'GcaDef', 'Tkl', 'TklWon', 'TklDef3rd', 'TklMid3rd', 'TklAtt3rd', 'TklDri', 'TklDriAtt', 'TklDri%', 'TklDriPast

In [42]:
# stampa le rows di comp con Serie A e mi selezioni solo la colonna goals e player
df.loc[df['Comp'] == 'Serie A', ['Shots', 'Player']]

,Shots,Player
2833,0.00,C. Tătărușanu
517,1.43,Samu Castillejo
2489,0.76,A. Romagnoli
2411,3.03,A. Rebić
1015,0.17,M. Gabbia
...,...,...
3125,0.00,Alessandro Zanoli
3126,0.00,Mattia Zanotti
3148,1.02,Nadir Zortea
3151,0.00,Petar Zovko


In [43]:
# remove null and duplicate values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True, subset=['Player', 'Rk'])
# print a Player that includes González in the Player
print(df[df['Player'].str.contains('González')])

        Rk          Player Nation Pos     Comp   Age  Born  MP  Starts   Min  \
1114  1035     G. González    URU  DF  La Liga  27.0  1994   9       3   365   
1119  1036     N. González    ESP  MF  La Liga  20.0  2002  27      12  1114   
1122  1037     N. González    ARG  FW  Serie A  24.0  1998  33      26  2357   
1111  1033  Diego González    ESP  DF  La Liga  27.0  1995  28      19  1793   

      ...  PKwon  PKcon   OG  Recov  AerWon  AerLost  AerWon%  \
1114  ...   0.00   0.00  0.0   8.54    2.44     2.93     45.5   
1119  ...   0.00   0.00  0.0   7.50    0.65     0.97     40.0   
1122  ...   0.11   0.00  0.0   4.58    3.13     2.52     55.4   
1111  ...   0.00   0.05  0.0   6.83    1.71     1.71     50.0   

                          Squad  \
1114      Club Atlético Peñarol   
1119  Club Atlético River Plate   
1122  Club Atlético River Plate   
1111                  Málaga CF   

                                          PlayerFaceUrl  \
1114  https://cdn.sofifa.net/players/2

In [44]:
print(df.describe().transpose())


          count         mean         std     min      25%      50%      75%  \
Rk       2329.0  1460.254186  840.756215     1.0   725.00  1456.00  2196.00   
Age      2329.0    26.986260    4.238871    18.0    24.00    27.00    30.00   
Born     2329.0  1994.510090    4.234635  1981.0  1992.00  1995.00  1998.00   
MP       2329.0    20.594676   11.053280     1.0    11.00    22.00    30.00   
Starts   2329.0    15.440103   11.236101     0.0     5.00    15.00    25.00   
...         ...          ...         ...     ...      ...      ...      ...   
OG       2329.0     0.003744    0.021510     0.0     0.00     0.00     0.00   
Recov    2329.0     7.531816    3.268692     0.0     5.29     7.50     9.44   
AerWon   2329.0     1.670593    1.789465     0.0     0.58     1.29     2.31   
AerLost  2329.0     1.805500    2.039547     0.0     0.89     1.40     2.13   
AerWon%  2329.0    41.905668   22.821040     0.0    30.30    44.70    58.00   

            max  
Rk       2921.0  
Age        41.0

In [45]:
# give me values of CarDist of this player position ["MF", "MFDF", "MFFW"]
print(df.loc[(df['Pos'] == "MF") | (df['Pos'] == "MFDF") | (df['Pos'] == "MFFW"), 'CarDis'])

839     2.12
2425    1.52
2857    2.12
1427    1.09
2717    0.99
        ... 
2100    0.54
727     0.37
2073    0.78
650     0.50
1613    0.93
Name: CarDis, Length: 724, dtype: float64


In [ ]:
print(df[df.duplicated(['Player'], keep=False)][['Player', 'Squad', 'Pos', 'Goals', '90s']])
df = df.sort_values('90s', ascending=False).drop_duplicates(['Player'])
print(df[df.duplicated(['Player'], keep=False)][['Player', 'Squad', 'Pos', 'Goals', '90s']])
df.sort_values(by=['Player']).reset_index(drop=True)

In [48]:
competitions = ["Serie A", "Premier League", "La Liga", "Bundesliga", "Ligue 1"]

forward_positions = ["FW", "FWMF", "FWDF"]
midfield_positions = ["MF", "MFDF", "MFFW"]
defender_positions = ["DF", "DFMF", "DFFW"]

common_features = ["Player", "Squad", "Comp", "MP", "Pos", "90s", "PlayerFaceUrl", "ClubLogoUrl"]

forward_features = ["Goals", "Shots", "SoT", "G/Sh", "G/SoT", "ShoDist", "GCA", "SCA", "Off", "PKwon", "ScaDrib", "Assists",
                    "ScaPassLive", "Car3rd", "ScaFld",  "Carries", "CarTotDist", "CarPrgDist", 'CPA', "CarMis", "CarDis"]  # "ToAtt", "ToSuc"

midfielder_features = ["Goals", "PasTotCmp", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", "Assists", "PasAss", "Pas3rd", "Crs", "PasCmp",
                       "PasOff", "PasBlocks", "SCA", "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "GCA", "GcaPassLive",
                       "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "Tkl", "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri",
                       "TklDriAtt", "TklDri%", "TklDriPast", "Blocks", "BlkSh", "Int", "Recov", "Carries", "CarTotDist", "CarPrgDist", "Fld"]

defender_features = ["PasTotCmp", "PasTotDist", "PasTotPrgDist", "Tkl", "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDriPast", "Blocks",
                     "BlkSh", "Int", "Tkl+Int", "Recov", "AerWon", "AerLost", "Carries", "CarTotDist", "CarPrgDist", "CrdY", "CrdR", "Fls", "Clr"]


positions = [forward_positions,midfield_positions,defender_positions]

In [49]:
# removing outliers, who never player more than 180 mins
import os
dataset_path = './data/dataset-preproc'

df = df[df['90s'] > 5]

if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)

for position in positions:
    pos, features_set = None, None
    
    if position == forward_positions:
        pos = "forward"
        features_set = common_features + forward_features
    elif position == midfield_positions:
        pos = "midfielder"
        features_set = common_features + midfielder_features
    else:
        pos = "defender"
        features_set = common_features + defender_features

    df_refined_1 = df[df["Pos"] == position[0]][features_set]
    df_refined_2 = df[df["Pos"] == position[1]][features_set]
    df_refined_3 = df[df["Pos"] == position[2]][features_set]
    df_forward = pd.concat([df_refined_1, df_refined_2, df_refined_3], axis=0)
    df_forward.to_csv(f'{dataset_path}/{pos}.csv', index=False)
        